# Vacuum Adapters

## Cyclone Adapters

In [1]:
import cadquery as cq
from jupyter_cadquery import (
    PartGroup, Part, Edges, Faces, Vertices, show,
    close_viewer, close_viewers, get_viewer, open_viewer, set_defaults, get_defaults
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay, reset_replay

Overwriting auto display for cadquery Workplane and Shape


In [2]:
set_defaults(axes=True, timeit=False)

In [3]:
cv = open_viewer("Adapters", cad_width=1200, height=1200, glass=True)

In [4]:
enable_replay(False, False)
show_object = replay


Enabling jupyter_cadquery replay


In [5]:
cyclone_inner_diameter = 50.75
cyclone_outer_diameter = 65.2
vacuum_hose_screw_pitch = 5 # mm per revolution
vacuum_hose_screw_height = 3 - 0.1
vacuum_hose_screw_width = 1.2
vacuum_hose_screw_angle = 0
vacuum_hose_outer_diameter = 39.5

In [6]:
def make_hose_helix_connecter(height):
    p = -vacuum_hose_screw_pitch
    h = height

    # Helix
    wire = cq.Wire.makeHelix(pitch=p, height=h, radius=vacuum_hose_outer_diameter)
    helix = cq.Workplane(obj=wire)
    return (
        cq.Workplane("XZ")  # helix is moving up the Z axis
        .center(vacuum_hose_outer_diameter / 2, 0)  # offset isosceles trapezoid
        .polyline(((-vacuum_hose_screw_height, vacuum_hose_screw_width / -3), (0.0, - vacuum_hose_screw_width), (0, vacuum_hose_screw_width), (-vacuum_hose_screw_height,  vacuum_hose_screw_width / 3)))
        .close()  # make edges a wire
        .sweep(helix, isFrenet=True)  # Frenet keeps orientation as expected
    )

show_object(make_hose_helix_connecter(40))


Use the multi select box below to select one or more steps you want to examine


In [7]:
def make_coupler(height):
    mantel_width = (cyclone_inner_diameter - vacuum_hose_outer_diameter) / 2
    rectangle_length = height
    stopper_width = (cyclone_outer_diameter - vacuum_hose_outer_diameter) / 2
    angle_degrees = 360.0
    
    helix = make_hose_helix_connecter(height)

    return helix + (
        cq.Workplane("XZ")
        .center(vacuum_hose_outer_diameter / 2, 0)
        .rect(mantel_width, rectangle_length, False)
        .polyline(((0, 0), (mantel_width, 0), (mantel_width, -10), (0, -vacuum_hose_screw_width)))
        .close()
        .center(0, height + 5)
        .rect(stopper_width, -5, False)
        .center(0, -(height + 5))
        .center(-vacuum_hose_outer_diameter / 2, 0)
        .revolve()
        .fillet(1.0)
    )

# Displays the result of this script
coupler = make_coupler(20)
show_object(coupler)

cq.exporters.export(coupler, "coupler.stl")

Use the multi select box below to select one or more steps you want to examine


## Saw Adapter

In [8]:
saw_guard_width = 70
saw_guard_height = 120

wall_thickness = 8

hose_insert_outer_diameter = 51
hose_insert_diameter = 45
hose_clip_hole_height = 5.5
hose_clip_hole_width = 10.5
hose_clip_hole_depth = 11.6 + hose_clip_hole_height / 2

hose_insert_depth = 24.7
hose_insert_end_diameter = 32

In [29]:
path_length = 50

path = cq.Workplane("XZ").moveTo(0, 0).lineTo(path_length, 0)

recttocircleSweep = (
    cq.Workplane("YZ")
    .workplane(offset=0.0)
    .rect(saw_guard_width + wall_thickness, saw_guard_height + wall_thickness)
    .workplane(offset=path_length * 0.1)
    .rect(saw_guard_width + wall_thickness - 1, saw_guard_height + wall_thickness - 1)
    .workplane(offset=path_length * 0.1)
    .rect(saw_guard_width + wall_thickness - wall_thickness, saw_guard_height + wall_thickness - wall_thickness)
    .workplane(offset=path_length * 0.5)
    .circle(hose_insert_outer_diameter / 2 + 3.0)
    .workplane(offset=path_length * 0.2)
    .circle(hose_insert_outer_diameter / 2 + 0.5)
    .workplane(offset=path_length * 0.1)
    .circle(hose_insert_outer_diameter / 2)
    .sweep(path, multisection=True)
)

innerrecttocircleSweep = (
    cq.Workplane("YZ")
    .workplane(offset=0.0)
    .rect(saw_guard_width, saw_guard_height)
    .workplane(offset=path_length * 0.1)
    .rect(saw_guard_width - 1.5, saw_guard_height - 1.5)
    .workplane(offset=path_length * 0.1)
    .rect(saw_guard_width -  wall_thickness, saw_guard_height - wall_thickness)
    .workplane(offset=path_length * 0.6)
    .circle(hose_insert_end_diameter / 2 + 2)
    .workplane(offset=path_length * 0.1)
    .circle(hose_insert_end_diameter / 2 + 0.7)
    .workplane(offset=path_length * 0.1)
    .circle(hose_insert_end_diameter / 2)
    .sweep(path, multisection=True, isFrenet=True)
)

funnel = recttocircleSweep - innerrecttocircleSweep

adapter = (
    funnel
    .faces('>X')
    .circle(hose_insert_outer_diameter / 2)
    .circle(hose_insert_diameter / 2)
    .extrude(hose_insert_depth)
    .faces('>X')
    .workplane()
    .transformed(rotate=(0, 90, 0))
    .workplane()
    .move(hose_clip_hole_depth,0)
    .ellipse(hose_clip_hole_height / 2, hose_clip_hole_width / 2)
    .cutThruAll()
)

show_object(adapter)

cq.exporters.export(adapter, "suction-funnel.stl")

Use the multi select box below to select one or more steps you want to examine
